In [ ]:
# Mouse actions
import cv2
import time
import numpy as np
import HandTrackingModule as htm
import math
from comtypes import CLSCTX_ALL
from pycaw.pycaw import AudioUtilities, IAudioEndpointVolume
import pyautogui

wCam, hCam = 640, 480
cap=cv2.VideoCapture(0)
cap.set(3,wCam)
cap.set(4,hCam)
pTime = 0
frameR=100
smoothening=12
plocX,plocY=0,0
clocX,clocY=0,0


detector=htm.HandDetector(maxHands=1)
wScreen,hScreen=pyautogui.size()
pyautogui.FAILSAFE=False

while True:
    success,img=cap.read()
    img = cv2.flip(img, 1)
    img=detector.findHands(img)
    lmList,bbox=detector.findPosition(img)
    
    if len(lmList)!=0:
        [x1,y1]=lmList[8][1:]
        [x2,y2]=lmList[12][1:]
        fingers=detector.fingersUp()
        cv2.rectangle(img,(frameR,frameR),(wCam-frameR,hCam-frameR),(255,0,255),2)
        
        # For hovering
        if fingers[1]==1 and fingers[2]==0:
            
            x3=np.interp(x1,(frameR,wCam-frameR),(0,wScreen))
            y3=np.interp(y1,(frameR,hCam-frameR),(0,hScreen))
            
            clocX=plocX+(x3-plocX)/smoothening
            clocY=plocY+(y3-plocY)/smoothening
            
            pyautogui.moveTo(clocX,clocY)
            plocX,plocY=clocX,clocY
        
        # Left click functionality
        
        if fingers[1]==1 and fingers[2]==1:
            length,img,lineInfo=detector.findDistance(8,12,img)
            # print(length)
            if length<40:
                pyautogui.click()
        
        # Design for right click
        if fingers[1]==1 and fingers[2]==0:
            if fingers[4]==1:
                pyautogui.rightClick()
                
        
    
    cTime = time.time()
    fps = 1/(cTime-pTime)
    pTime = cTime

    cv2.putText(img, f'FPS:{int(fps)}', (48, 70),
                cv2.FONT_HERSHEY_PLAIN, 2, (255, 0, 0), 3)
    cv2.imshow("Image",img)
    cv2.waitKey(1)

In [ ]:
# volume control gesture --- Use right hand for the volume control
import cv2
import time
import numpy as np
import HandTrackingModule as htm
from comtypes import CLSCTX_ALL
from pycaw.pycaw import AudioUtilities, IAudioEndpointVolume

wCam, hCam = 640, 480
# boundingBox=[]
cap = cv2.VideoCapture(0)
cap.set(3, wCam)
cap.set(4, hCam)
pTime = 0


detector = htm.HandDetector(maxHands=1)

devices = AudioUtilities.GetSpeakers()
interface = devices.Activate(
    IAudioEndpointVolume._iid_, CLSCTX_ALL, None)
volume = interface.QueryInterface(IAudioEndpointVolume)
volumeRange = volume.GetVolumeRange()
minimumVolume = volumeRange[0]
maximumVolume = volumeRange[1]
area = 0
while True:
    success, img = cap.read()
    img = cv2.flip(img, 1)  # Flipping is needed to create mirroring
    # Find the hands
    img,which_hand=detector.findHands(img)
    lmList, boundingBox = detector.findPosition(img, handNo=0, draw=True)
    if which_hand=="Right":
        if len(lmList) != 0:
        # Filter based on size
            area = (boundingBox[2]-boundingBox[0]) * \
                (boundingBox[3]-boundingBox[1])//100
            # print(area)
            if 100 < area < 900:

                # Find the distance btwn index and thumb
                length, img, lineInfo = detector.findDistance(4, 8, img=img)

                # Converting length to volume
                vol = np.interp(length, [10, 150], [0, 100])

                # Reduce resolution to make smoother.
                smoothness = 5
                vol = smoothness*round(vol/smoothness)

                # Check fingers which are up
                fingers=detector.fingersUp()
            

                # if pinky is down then set volume
                if not fingers[4]:
                    volume.SetMasterVolumeLevelScalar(vol/100, None)
                
            

    cTime = time.time()
    fps = 1/(cTime-pTime)
    pTime = cTime

    cv2.putText(img, f'FPS:{int(fps)}', (48, 70),
                cv2.FONT_HERSHEY_PLAIN, 2, (255, 0, 0), 3)
    cv2.imshow("Image", img)
    cv2.waitKey(1)

In [5]:
import cv2
import time
import numpy as np
import HandTrackingModule as htm
import screen_brightness_control as sbc

wCam, hCam = 640, 480
# boundingBox=[]
cap = cv2.VideoCapture(0)
cap.set(3, wCam)
cap.set(4, hCam)
pTime = 0


detector = htm.HandDetector(maxHands=1)
area = 0
while True:
    success, img = cap.read()
    img = cv2.flip(img, 1)  # Flipping is needed to create mirroring
    # Find the hands
    img,which_hand=detector.findHands(img)
    lmList, boundingBox = detector.findPosition(img, handNo=0, draw=True)
    if which_hand=="Left":
        if len(lmList) != 0:
        # Filter based on size
            area = (boundingBox[2]-boundingBox[0]) * \
                (boundingBox[3]-boundingBox[1])//100
            # print(area)
            if 100 < area < 900:

                # Find the distance btwn index and thumb
                length, img, lineInfo = detector.findDistance(4, 8, img=img)

                # Converting length to volume
                brightness = np.interp(length, [10, 150], [0, 100])

                # Reduce resolution to make smoother.
                smoothness = 5
                brightness = smoothness*round(brightness/smoothness)

                # Check fingers which are up
                fingers=detector.fingersUp()
            

                # if pinky is down then set volume
                if not fingers[4]:
                    sbc.set_brightness(brightness)

    cTime = time.time()
    fps = 1/(cTime-pTime)
    pTime = cTime

    cv2.putText(img, f'FPS:{int(fps)}', (48, 70),
                cv2.FONT_HERSHEY_PLAIN, 2, (255, 0, 0), 3)
    cv2.imshow("Image", img)
    cv2.waitKey(1)

KeyboardInterrupt: 

In [ ]:
## Sample use only..Not working

# Importing Libraries
import cv2
import mediapipe as mp
import time
import numpy as np
import HandTrackingModule as htm
import math
from comtypes import CLSCTX_ALL
from pycaw.pycaw import AudioUtilities, IAudioEndpointVolume

# Used to convert protobuf message to a dictionary.
from google.protobuf.json_format import MessageToDict

# Initializing the Model
mp_holistic = mp.solutions.holistic
holistic_model = mp_holistic.Holistic(
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5
)
mpHands = mp.solutions.hands
hands = mpHands.Hands(
    static_image_mode=False,
    model_complexity=1,
    min_detection_confidence=0.75,
    min_tracking_confidence=0.75,
    max_num_hands=2)
mpDraw = mp.solutions.drawing_utils

# Start capturing video from webcam
cap = cv2.VideoCapture(0)

while True:
    # Read video frame by frame
    success, img = cap.read()

    # Flip the image(frame)
    img = cv2.flip(img, 1)

    # Convert BGR image to RGB image
    imgRGB = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    # Process the RGB image
    results = hands.process(imgRGB)

    # If hands are present in image(frame)
    if results.multi_hand_landmarks:

        # Both Hands are present in image(frame)
        if len(results.multi_handedness) == 2:
            # Display 'Both Hands' on the image
            cv2.putText(img, 'Both Hands', (250, 50),
                        cv2.FONT_HERSHEY_COMPLEX,
                        0.9, (0, 255, 0), 2)

        # If any hand present
        else:
            for i in results.multi_handedness:

                # Return whether it is Right or Left Hand
                label = MessageToDict(i)['classification'][0]['label']

                if label == 'Left':

                    # Display 'Left Hand' on
                    # left side of window
                    cv2.putText(img, label+' Hand',
                                (20, 50),
                                cv2.FONT_HERSHEY_COMPLEX,
                                0.9, (0, 255, 0), 2)

                if label == 'Right':

                    # Display 'Left Hand'
                    # on left side of window
                    cv2.putText(img, label+' Hand', (460, 50),
                                cv2.FONT_HERSHEY_COMPLEX,
                                0.9, (0, 255, 0), 2)
                    # Draw landmark points
                    
                    
                    

    # Display Video and when 'q'
    # is entered, destroy the window
    cv2.imshow('Image', img)
    if cv2.waitKey(1) & 0xff == ord('q'):
        break